In [1]:
import rasterio
import os
import numpy as np
import cv2
import glob
import collections

%matplotlib inline
import matplotlib.pyplot as plt

import sys
sys.path.append('../../GranularData/')

from utils import match

In [2]:
path1 = '../datasets/raipur/S2A_MSIL1C_20170101T050212_N0204_R119_T44QNJ_20170101T050737.SAFE'
safes = os.listdir('../datasets/raipur/')

safes_map = {}
for safe in safes:
    if 'SAFE' in safe and 'R119' in safe:
        date = safe.split('_')[2]
        safes_map[date] = safe
    
safes_sorted = collections.OrderedDict(sorted(safes_map.items()))

In [3]:
def stretch_8bit(bands, lower_percent=5, higher_percent=95):
    """stretch_8bit takes a 3 band image (as an array) and returns an 8bit array with clipped values (5-95%) stretched to 0-255

    Parameters
    ----------
    bands : numpy.array
        Numpy array of shape  (*,*,3)
    lower_percent : type
        Lower threshold below which array values will be discarded (the default is 5).
    higher_percent : type
        Upper threshold above which array values will be discarded (the default is 95).

    Returns
    -------
    numpy.array
        Numpy array containing np.uint8 values of shape bands.shape

    """
    out = np.zeros_like(bands)
    for i in range(3):
        a = 0
        b = 255
        real_values = bands[:,:,i].flatten()
        real_values = real_values[real_values > 0]
        c = np.percentile(real_values, lower_percent)
        d = np.percentile(real_values, higher_percent)
        t = a + (bands[:,:,i] - c) * (b - a) / (d - c)
        t[t<a] = a
        t[t>b] = b
        out[:,:,i] =t
    return out.astype(np.uint8)

In [4]:
for safe in safes:
    if 'SAFE' in safe and 'R119' in safe:
        date = safe.split('_')[2]
        
        base_name = glob.glob('../datasets/raipur/' + safe + "/GRANULE/**/IMG_DATA/**.jp2")[0][:-7]
        b02 = rasterio.open(base_name + 'B02.jp2')
        b03 = rasterio.open(base_name + 'B03.jp2')
        b04 = rasterio.open(base_name + 'B04.jp2')
        
        r = b02.read()[0]
        g = b03.read()[0]
        b = b04.read()[0]
        
        rgb = cv2.merge([r,g,b])
        
        rgb_scaled = stretch_8bit(rgb, 2, 98)
        
        cv2.imwrite('../datasets/raipur_pngs_crop/' + date + '.png', rgb_scaled[5100:7000, 7200:8800, :])